In [ ]:
import pandas as pd
import numpy as np
import recordlinkage
from recordlinkage.index import Block
import networkx as nx
from collections import defaultdict
import time

start = time.time()

In [ ]:
df = pd.read_csv("../data/customers_raw.csv")
print(f"Loaded {len(df):,} records")
df.head()

### Data Processing

In [ ]:
df_clean = df.copy()

df_clean['first_name_clean'] = df_clean['first_name'].str.lower().str.strip()
df_clean['last_name_clean'] = df_clean['last_name'].str.lower().str.strip()
df_clean['email_username'] = df_clean['email'].str.split('@').str[0].str.lower()
df_clean['phone_clean'] = df_clean['phone'].astype(str).str.replace('-', '').str.replace('None', '')

df_clean['block_key'] = (
    df_clean['last_name_clean'].str[:3] + '_' + 
    df_clean['first_name_clean'].str[:1] + '_' + 
    df_clean['province'].fillna('')
)

print(f"Created {df_clean['block_key'].nunique():,} blocks")

In [ ]:
block_sizes = df_clean['block_key'].value_counts()
block_sizes.describe()

### Cell blocking and candidate pair generation


In [ ]:
df_clean = df_clean.set_index('record_id')

block_sizes = df_clean['block_key'].value_counts()
MAX_BLOCK_SIZE = 2000

valid_blocks = block_sizes[block_sizes <= MAX_BLOCK_SIZE].index.tolist()
df_filtered = df_clean[df_clean['block_key'].isin(valid_blocks)]

indexer = recordlinkage.Index()
indexer.block('block_key')
candidate_pairs = indexer.index(df_filtered)

print(f"Valid blocks: {len(valid_blocks):,} | Records: {len(df_filtered):,} | Candidate pairs: {len(candidate_pairs):,}")

### Similarity comparison

In [ ]:
compare = recordlinkage.Compare()
compare.exact('phone_clean', 'phone_clean', label='phone_exact')
compare.exact('email_username', 'email_username', label='email_exact')
compare.string('first_name_clean', 'first_name_clean', method='jarowinkler', label='first_name_sim')
compare.string('last_name_clean', 'last_name_clean', method='jarowinkler', label='last_name_sim')
compare.exact('date_of_birth', 'date_of_birth', label='dob_exact')

CHUNK_SIZE = 1_000_000
all_features = []

for i in range(0, len(candidate_pairs), CHUNK_SIZE):
    chunk_pairs = candidate_pairs[i:i+CHUNK_SIZE]
    chunk_features = compare.compute(chunk_pairs, df_filtered)
    all_features.append(chunk_features)

features = pd.concat(all_features)
del all_features

print(f"Computed {len(features):,} comparisons")
features.describe()

### Classify Matches

In [ ]:
features['total_score'] = (
    features['phone_exact'] * 3.0 +
    features['email_exact'] * 3.0 +
    features['first_name_sim'] * 1.0 +
    features['last_name_sim'] * 1.0 +
    features['dob_exact'] * 2.0
)

THRESHOLD = 7.0
matches = features[features['total_score'] >= THRESHOLD]

print(f"Found {len(matches):,} matches (threshold={THRESHOLD})")

### Build graph and find clusters

In [ ]:
G = nx.Graph()
G.add_nodes_from(df_filtered.index)
G.add_edges_from(matches.index.tolist())

components = list(nx.connected_components(G))
print(f"Edges: {G.number_of_edges():,} | Unique customers: {len(components):,}")

In [ ]:
import matplotlib.pyplot as plt

interesting_clusters = [c for c in components if 2 <= len(c) <= 10][:5]

fig, axes = plt.subplots(1, min(5, len(interesting_clusters)), figsize=(15, 4))

if len(interesting_clusters) == 1:
    axes = [axes]

for idx, cluster in enumerate(interesting_clusters):
    subgraph = G.subgraph(cluster)
    ax = axes[idx]
    pos = nx.spring_layout(subgraph, seed=42)
    nx.draw(subgraph, pos, ax=ax, with_labels=True, node_color='lightblue',
            node_size=500, font_size=6, font_weight='bold', edge_color='gray')
    ax.set_title(f"Cluster {idx+1} ({len(cluster)} records)")

plt.tight_layout()
plt.show()

### Assign resolved IDs

In [ ]:
record_to_cluster = {}
for cluster_id, component in enumerate(components):
    for record_id in component:
        record_to_cluster[record_id] = f"CUST_{cluster_id:07d}"

df_filtered = df_filtered.copy()
df_filtered['resolved_customer_id'] = df_filtered.index.map(record_to_cluster)

print(f"Resolved {len(df_filtered):,} records to {df_filtered['resolved_customer_id'].nunique():,} customers")

In [ ]:
df_truth_indexed = df_truth.set_index('record_id')
df_filtered['true_customer_id'] = df_truth_indexed.loc[df_filtered.index, 'true_customer_id']

purity_check = df_filtered.groupby('resolved_customer_id')['true_customer_id'].nunique()
perfect = (purity_check == 1).sum()
total_clusters = len(purity_check)

frag_check = df_filtered.groupby('true_customer_id')['resolved_customer_id'].nunique()
fragmented = (frag_check > 1).sum()

print(f"Purity: {perfect:,}/{total_clusters:,} ({perfect/total_clusters*100:.1f}%) | Fragmented: {fragmented:,}")

### Save the clean data

In [ ]:
output_df = df_filtered.reset_index()
output_df.to_csv('../data/customer_data_cleaned.csv')

print(f"Saved: {len(output_df):,} records, {output_df['resolved_customer_id'].nunique():,} customers")